In [1]:
import pandas as pd
import sqlite3
import os

csv_path = "../Processed/jobs_enriched.csv"
db_path = "../Sql/jobs.db"

os.makedirs("../Sql", exist_ok=True)

df = pd.read_csv(csv_path)
conn = sqlite3.connect(db_path)

df.to_sql("jobs", conn, if_exists="replace", index=False)

conn.execute("SELECT COUNT(*) FROM jobs").fetchone()


(118851,)

In [2]:
pd.read_sql("PRAGMA table_info(jobs);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,job_title,TEXT,0,None,0
1,1,job_description,TEXT,0,None,0
2,2,location,TEXT,0,None,0
3,3,is_entry_level,INTEGER,0,None,0
4,4,min_experience_years,REAL,0,None,0
5,5,experience_bucket,TEXT,0,None,0


In [3]:
pd.read_sql("""
SELECT
  COUNT(*) AS total_jobs,
  SUM(is_entry_level) AS entry_level_jobs
FROM jobs;
""", conn)


,total_jobs,entry_level_jobs
0,118851,9810


In [4]:
pd.read_sql("""
SELECT
  ROUND(
    100.0 * SUM(CASE WHEN min_experience_years >= 3 THEN 1 ELSE 0 END) / COUNT(*),
    2
  ) AS pct_entry_requires_3plus
FROM jobs
WHERE is_entry_level = 1
  AND min_experience_years IS NOT NULL;
""", conn)


,pct_entry_requires_3plus
0,40.37


In [5]:
pd.read_sql("""
SELECT
  experience_bucket,
  COUNT(*) AS job_count
FROM jobs
WHERE is_entry_level = 1
  AND min_experience_years IS NOT NULL
GROUP BY experience_bucket
ORDER BY job_count DESC;
""", conn)


,experience_bucket,job_count
0,0–1 years,1358
1,2–3 years,1015
2,5+ years,584
3,4–5 years,310


In [6]:
pd.read_sql("""
SELECT
  CASE
    WHEN min_experience_years IS NULL THEN 'Not specified'
    ELSE 'Specified'
  END AS experience_disclosure,
  COUNT(*) AS job_count
FROM jobs
WHERE is_entry_level = 1
GROUP BY experience_disclosure;
""", conn)


,experience_disclosure,job_count
0,Not specified,6543
1,Specified,3267


In [7]:
pd.read_sql("""
SELECT
  job_title,
  COUNT(*) AS postings
FROM jobs
WHERE is_entry_level = 1
  AND min_experience_years >= 4
GROUP BY job_title
ORDER BY postings DESC
LIMIT 15;
""", conn)


,job_title,postings
0,retail sales associate,46
1,part-time sales associate - famous footwear,31
2,retail stocking associate,28
3,sales associate,25
4,lead sales associate,14
5,part-time retail sales associate,11
6,senior retail sales associate,8
7,litigation associate,8
8,"associate, part time sales",8
9,"principal associate, cloud authentication engi...",7


In [8]:
pd.read_sql("""
SELECT
  location,
  ROUND(AVG(min_experience_years), 2) AS avg_required_years,
  COUNT(*) AS postings
FROM jobs
WHERE is_entry_level = 1
  AND min_experience_years IS NOT NULL
GROUP BY location
HAVING postings >= 20
ORDER BY avg_required_years DESC
LIMIT 15;
""", conn)


,location,avg_required_years,postings
0,"raleigh, nc",3253.55,20
1,"san diego, ca",2276.64,22
2,"chicago, il",768.51,81
3,"houston, tx",9.11,37
4,"miami, fl",4.30,40
5,"san francisco, ca",4.17,35
6,"las vegas, nv",4.14,21
7,"new york, ny",4.11,189
8,"atlanta, ga",4.10,42
9,"new york, united states",4.09,33
